In [1]:
from utils import load_data, check_gpu, check_dir
from clean_session import preprocess_sessions
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
from functools import partial
import matplotlib.pyplot as plt
from clean_session import preprocess_sessions
from create_model_inputs import prepare_data, compute_session_fts, create_model_inputs
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2
def fprint(df, name):
    print(f'{name} shape: ({df.shape[0]:,}, {df.shape[1]})')
import pprint
pp = pprint.PrettyPrinter(indent=1)

In [2]:
nrows = 1000000
train = prepare_data('train', convert_action_type=False, nrows=nrows, recompute=False)
df = compute_session_fts(train, 'train')

[05-23 18:48:27 - utils-107 - load_data - INFO] Loading train using 1,000,000 rows (999,995 trimmed) which is 6.28% out of total train data
[05-23 18:48:27 - create_model_inputs-17 - flogger - INFO] raw train shape: (999,995, 12)
[05-23 18:48:27 - clean_session-56 - preprocess_sessions - INFO] Load from existing file: ./gbm_cache/preprocessed_train.snappy
[05-23 18:48:32 - create_model_inputs-66 - prepare_data - INFO] Sort df by user_id, session_id, timestamp, step
[05-23 18:48:39 - create_model_inputs-17 - flogger - INFO] Prepared train data shape: (5,200,153, 9)


KeyboardInterrupt: 

In [ ]:
df = df.groupby('session_id').last().reset_index()
# df['session_id_size'] = np.log(df['session_id_size'])
# df['cfs'] = df['current_filters_last_filters'].str.lower().str.split('|')

# df['prices'] = df['prices'].str.split('|')
# df['prices'] = df['prices'].apply(lambda x: [int(p) for p in x])

# df['time_steps'] = df['prices'].str.len()
# padding_mask = df['time_steps'] < 25
# df.loc[padding_mask, 'prices'] = df.loc[padding_mask, 'prices'].apply(lambda x: np.pad(x, (0, 25-len(x)),
#                                                                                        mode='constant'))
df['imps'] = df['impressions'].str.split('|')
df['imps'] = df['imps'].apply(lambda x: [int(i) for i in x])
# df.loc[padding_mask, 'impressions'] = (df.loc[padding_mask, 'impressions']
#                                          .apply(lambda x: np.pad(x, (0, 25-len(x)),mode='constant')))
df['reference'] = df['reference'].astype(int)
# filter out nan rows with reference_id not in impressions list, since if the true target in test
# is not in the impression list then it would not get evaluated
def assign_target(row):
    ref = row['reference']
    imp = list(row['imps'])
    if ref in imp:
        return imp.index(ref)
    else:
        return np.nan

df['target'] = df.apply(assign_target, axis=1)
# drop the ones whose reference is not in the impression list
df = df[df['target'].notna()].reset_index(drop=True)
df['target'] = df['target'].astype(int)
# def assign_last_ref_id(row, divide=True):
#     ref = row['reference_last_reference_id']
#     # although reference_id got converted to int, but the reference_last_reference_id was calculated
#     # when it was still str value, so here we look up the index in str of impressions
#     imp = [str(i) for i in row['imps']]

#     if pd.isna(ref):
#         return np.nan
#     else:
#         if ref in imp:
#             if divide:
#                 return (imp.index(ref) + 1) / len(imp)
#             else:
#                 return imp.index(ref) + 1
#         else:
#             return np.nan
# assign_last_ref_id_func = partial(assign_last_ref_id, divide=False)
# df['last_ref_ind'] = df.apply(assign_last_ref_id_func, axis=1)

### comapre session with size 1 and the rest

In [ ]:
df.head(2)

In [ ]:
ones = df[df['session_id_size']==1].reset_index(drop=True)
multiples = df[df['session_id_size']>1].reset_index(drop=True)

In [ ]:
oids = ones['session_id'].unique()
mids = multiples['session_id'].unique()

In [ ]:
df[df.session_id=='0000059a39020']

---
### ones

In [ ]:
display_cols = ['session_id', 'step', 'action_type', 'current_filters', 'reference', 'timestamp',
                'impressions', 'prices', 'target', 'session_id_size']
raw_display_cols = [c for c in train.columns if c in display_cols]

In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
one_rid = np.random.choice(oids, 1)[0]
print(one_rid)
df[df.session_id==one_rid][display_cols].reset_index(drop=True)

In [ ]:
# df[df.session_id==one_rid][display_cols].reset_index(drop=True).to_dict(orient='list')

In [ ]:
from colorama import Fore
dic = df[df.session_id==one_rid][display_cols].reset_index(drop=True).to_dict(orient='list')
prices = dic['prices'][0].split('|')
ref = str(dic['reference'][0])
imps = dic['impressions'][0].split('|')
# prices[imps.index(ref)] = f'{Fore.RED} {prices[imps.index(ref)]}' #{test_id}
prices[imps.index(ref)] = f'==={prices[imps.index(ref)]}===' #{test_id}
dic['prices'] = '|'.join(prices)
pp.pprint(dic)
# print(dic['prices'])

In [ ]:
print(f'{Fore.RED}{prices[imps.index(ref)]}')

In [ ]:
# # and the raw
# train[train.session_id==one_rid][raw_display_cols].reset_index(drop=True)

In [ ]:
# pp.pprint(train[train.session_id==one_rid][raw_display_cols].reset_index(drop=True).to_dict(orient='list'))

---
### Longer

In [ ]:
mid_rid = np.random.choice(mids, 1)[0]
print(mid_rid)
# df[df.session_id==mid_rid][display_cols].reset_index(drop=True)

In [ ]:
np.sort([int(i) for i in df[df.session_id==mid_rid]['prices'].str.split('|').values[0]])

In [ ]:
# pp.pprint(df[df.session_id==mid_rid][display_cols].reset_index(drop=True).to_dict(orient='list'))

In [ ]:
# the raw
train[train.session_id==mid_rid][raw_display_cols].reset_index(drop=True)

In [ ]:
pp.pprint(train[train.session_id==mid_rid][raw_display_cols].reset_index(drop=True).to_dict(orient='list'))

In [ ]:
# dup_mask = train[[c for c in train.columns if c not in ['timestamp', 'step']]].duplicated(keep=False)
# dup_mask.sum()

In [ ]:
# dup_cols = [c for c in train.columns if c not in ['timestamp', 'step']]
# # train[dup_cols].loc[train[dup_cols].shift(-1) != train[dup_cols]]
# train.groupby(dup_cols).apply(lambda x: )

In [ ]:
train.shape

In [ ]:
99508/train.shape[0]

### Look at test set
---

In [ ]:
test = load_data('tedt')

---
### Click-encodings

In [ ]:
from create_model_inputs import click_view_encoding
_ = click_view_encoding(sids=None, fold='testing', m=5)

In [ ]:
cv_enc = pd.read_csv('./cache/foldtesting_clickview_encodings.csv')

In [ ]:
cv_enc.shape

In [ ]:
cv_enc['clicked'].hist(bins=100)

---
### clickouts

In [ ]:
%%time
df = load_data('train', usecols=['session_id', 'action_type', 'impressions', 'reference'])

In [ ]:
df.shape

In [ ]:
df = df.drop_duplicates(subset=[c for c in df.columns if c != 'step']).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
%%time
# filter away sessions do not have clickout
def filter_away(df):
    return (df.action_type=='clickout item').sum() > 0
fids = df.groupby('session_id').apply(filter_away)

In [ ]:
df_has = df[df.session_id.isin(fids[fids].index)]

In [ ]:
df_has.shape

In [ ]:
# number of clickout rows
(df_has.action_type == 'clickout item').sum()

In [ ]:
last_df_has = df_has.groupby('session_id').last().reset_index()

In [ ]:
last_df_has.shape

In [ ]:
# average number of clickout per session
ctns = df_has[df_has.action_type=='clickout item'].groupby('session_id')['session_id'].size()

In [ ]:
ctns.head()

In [ ]:
s = df_has.groupby('session_id')['session_id'].size()

In [ ]:
s.head()

In [ ]:
# avg_clickout = ctns/s

In [ ]:
# avg_clickout.hist(bins=100)

In [ ]:
np.log(ctns).hist(bins=100)

In [ ]:
ctn